# Simplified NeSy Framework Demo

A step-by-step simplified version of the NeSy (Neural-Symbolic) framework with:
- Finite lists
- Countably infinite lists (lazy)
- Real intervals with numerical integration

## Step 1: Basic Type Classes for Truth Values

In [ ]:
{-# LANGUAGE MultiParamTypeClasses, FlexibleInstances #-}
import qualified Data.Map as Map
import qualified Data.Set as Set
import Data.Maybe
import Data.List (foldl')

-- Algebra of truth values
class TwoMonBLat a where
  top, bot :: a
  neg :: a -> a
  conj, disj, implies :: a -> a -> a
  -- Default implementations
  neg a = a `implies` bot
  a `implies` b = neg a `disj` b

-- Simple Boolean instance
instance TwoMonBLat Bool where
  top = True
  bot = False
  neg = not
  conj = (&&)
  disj = (||)
  implies a b = not a || b

print "Truth values algebra defined"

: 

## Step 2: Domain Types

We define three types of domains:
- Finite lists
- Countably infinite lists (lazy)
- Real intervals

In [ ]:
-- Domain representation
data Domain a = 
    Finite [a]                    -- Finite list of elements
  | CountableInfinite [a]        -- Lazy infinite list
  | RealInterval Double Double    -- Real interval (a,b)
  deriving Show

-- Aggregation over domains
class Aggregate a where
  -- Existential quantification
  existsOver :: Domain a -> (a -> Bool) -> Bool
  -- Universal quantification  
  forallOver :: Domain a -> (a -> Bool) -> Bool

instance Aggregate Double where
  existsOver (Finite xs) pred = any pred xs
  existsOver (CountableInfinite xs) pred = any pred (take 10000 xs)  -- Limit for safety
  existsOver (RealInterval a b) pred = 
    let points = [a, a+0.01 .. b]  -- Sample points
    in any pred points
    
  forallOver (Finite xs) pred = all pred xs
  forallOver (CountableInfinite xs) pred = all pred (take 10000 xs)  -- Limit for safety
  forallOver (RealInterval a b) pred =
    let points = [a, a+0.01 .. b]  -- Sample points
    in all pred points

print "Domain types and aggregation defined"

## Step 3: Numerical Integration

Simple trapezoidal rule for numerical integration over real intervals.

In [ ]:
:set -package numeric-tools
print "numeric-tools enabled"

: 

In [ ]:
import Numeric.Tools.Integration (defQuad, quadTrapezoid, quadSimpson, quadRomberg, quadRes)

-- Simple trapezoidal integration
integrate :: (Double -> Double) -> Double -> Double -> Int -> Double
integrate f a b n = 
  let h = (b - a) / fromIntegral n
      points = [a + fromIntegral i * h | i <- [0..n]]
      values = map f points
      trapezoids = zipWith (\x y -> (x + y) / 2 * h) (init values) (tail values)
  in sum trapezoids

-- Provide higher-accuracy integration via numeric-tools

data IntegrationMethod = Trapezoid | Simpson | Romberg deriving (Show, Eq)

integrateOver :: (Double -> Double) -> (Double, Double) -> IntegrationMethod -> Double
integrateOver f (a,b) method = 
  case method of
    Trapezoid -> quadRes (quadTrapezoid defQuad (a,b) f)
    Simpson   -> quadRes (quadSimpson   defQuad (a,b) f)
    Romberg   -> quadRes (quadRomberg   defQuad (a,b) f)

-- Quick check: should be close to 1/3
let testIntegralRomberg = integrateOver (\x -> x*x) (0,1) Romberg
print $ "∫_0^1 x^2 dx ≈ " ++ show testIntegralRomberg ++ ", exact 1/3 ≈ " ++ show (1/3)

## Step 4: Syntax

Simplified syntax for NeSy formulas.

In [ ]:
type Ident = String

-- Terms
data Term = 
    Var Ident
  | Const Double
  | Appl Ident [Term]
  deriving Show

-- Formulas
data Formula =
    T | F
  | Pred Ident [Term]
  | Not Formula
  | And Formula Formula
  | Or Formula Formula
  | Implies Formula Formula
  | Forall Ident Formula
  | Exists Ident Formula
  deriving Show

print "Syntax defined"

## Step 5: Semantics

Interpretation and evaluation functions.

In [ ]:
-- Interpretation structure
data Interpretation a = Interpretation {
  domain :: Domain a,
  functions :: Map.Map Ident ([a] -> a),
  predicates :: Map.Map Ident ([a] -> Bool)
}

-- Valuation (variable assignment)
type Valuation a = Map.Map Ident a

-- Evaluate terms
evalTerm :: Interpretation a -> Valuation a -> Term -> a
evalTerm interp val (Var var) = 
  case Map.lookup var val of
    Just v -> v
    Nothing -> error $ "Variable " ++ var ++ " not found"
evalTerm interp val (Const c) = 
  case domain interp of
    RealInterval _ _ -> c  -- For real domains, constants are doubles
    _ -> error "Constants only supported for real domains"
evalTerm interp val (Appl f args) =
  let argVals = map (evalTerm interp val) args
      func = case Map.lookup f (functions interp) of
               Just fn -> fn
               Nothing -> error $ "Function " ++ f ++ " not found"
  in func argVals

-- Evaluate formulas
evalFormula :: Interpretation Double -> Valuation Double -> Formula -> Bool
evalFormula _ _ T = True
evalFormula _ _ F = False
evalFormula interp val (Pred p args) =
  let argVals = map (evalTerm interp val) args
      pred = case Map.lookup p (predicates interp) of
               Just pr -> pr
               Nothing -> error $ "Predicate " ++ p ++ " not found"
  in pred argVals
evalFormula interp val (Not f) = not (evalFormula interp val f)
evalFormula interp val (And f1 f2) = 
  evalFormula interp val f1 && evalFormula interp val f2
evalFormula interp val (Or f1 f2) = 
  evalFormula interp val f1 || evalFormula interp val f2
evalFormula interp val (Implies f1 f2) = 
  not (evalFormula interp val f1) || evalFormula interp val f2
evalFormula interp val (Forall var f) =
  forallOver (domain interp) $ \x -> 
    evalFormula interp (Map.insert var x val) f
evalFormula interp val (Exists var f) =
  existsOver (domain interp) $ \x -> 
    evalFormula interp (Map.insert var x val) f

print "Semantics defined"

## Step 6: Example - Finite Domain

Simple example with finite set of integers.

In [ ]:
-- Example interpretation: integers 1-6 (like dice)
diceInterp :: Interpretation Double
diceInterp = Interpretation {
  domain = Finite [1.0, 2.0, 3.0, 4.0, 5.0, 6.0],
  functions = Map.fromList [
    ("+", \[x,y] -> x + y),
    ("*", \[x,y] -> x * y)
  ],
  predicates = Map.fromList [
    ("==", \[x,y] -> x == y),
    (">", \[x,y] -> x > y),
    ("even", \[x] -> x `mod` 2 == 0)
  ]
}

-- Test some formulas
let formula1 = Exists "x" (Pred "even" [Var "x"])  -- ∃x. even(x)
let result1 = evalFormula diceInterp Map.empty formula1
print $ "∃x ∈ {1,2,3,4,5,6}. even(x) = " ++ show result1

let formula2 = Forall "x" (Pred ">" [Var "x", Const 0])  -- ∀x. x > 0
let result2 = evalFormula diceInterp Map.empty formula2
print $ "∀x ∈ {1,2,3,4,5,6}. x > 0 = " ++ show result2

let formula3 = Exists "x" (Pred "==" [Var "x", Const 7])  -- ∃x. x = 7
let result3 = evalFormula diceInterp Map.empty formula3
print $ "∃x ∈ {1,2,3,4,5,6}. x = 7 = " ++ show result3

## Step 7: Example - Real Interval

Example with real numbers and numerical integration.

In [ ]:
-- Example interpretation: real interval [0,10]
realInterp :: Interpretation Double
realInterp = Interpretation {
  domain = RealInterval 0.0 10.0,
  functions = Map.fromList [
    ("sin", \[x] -> sin x),
    ("cos", \[x] -> cos x),
    ("exp", \[x] -> exp x),
    ("+", \[x,y] -> x + y),
    ("*", \[x,y] -> x * y)
  ],
  predicates = Map.fromList [
    ("<", \[x,y] -> x < y),
    (">", \[x,y] -> x > y),
    ("==", \[x,y] -> abs (x - y) < 0.001),  -- Approximate equality
    ("positive", \[x] -> x > 0)
  ]
}

-- Test formulas on real interval
let realFormula1 = Exists "x" (Pred "positive" [Var "x"])  -- ∃x ∈ [0,10]. x > 0
let realResult1 = evalFormula realInterp Map.empty realFormula1
print $ "∃x ∈ [0,10]. x > 0 = " ++ show realResult1

let realFormula2 = Forall "x" (Pred "<" [Var "x", Const 15])  -- ∀x ∈ [0,10]. x < 15
let realResult2 = evalFormula realInterp Map.empty realFormula2
print $ "∀x ∈ [0,10]. x < 15 = " ++ show realResult2

let realFormula3 = Exists "x" (Pred "<" [Var "x", Const 5])  -- ∃x ∈ [0,10]. x < 5
let realResult3 = evalFormula realInterp Map.empty realFormula3
print $ "∃x ∈ [0,10]. x < 5 = " ++ show realResult3

## Step 8: Example - Countably Infinite Domain

Example with natural numbers (countably infinite).

In [ ]:
-- Example interpretation: natural numbers (countably infinite)
naturalsInterp :: Interpretation Double
naturalsInterp = Interpretation {
  domain = CountableInfinite [0.0, 1.0 ..],  -- 0, 1, 2, 3, ...
  functions = Map.fromList [
    ("+", \[x,y] -> x + y),
    ("*", \[x,y] -> x * y),
    ("succ", \[x] -> x + 1)
  ],
  predicates = Map.fromList [
    ("==", \[x,y] -> x == y),
    (">", \[x,y] -> x > y),
    ("<", \[x,y] -> x < y),
    ("even", \[x] -> x `mod` 2 == 0)
  ]
}

-- Test formulas on naturals
let natFormula1 = Exists "x" (Pred "even" [Var "x"])  -- ∃x ∈ ℕ. even(x)
let natResult1 = evalFormula naturalsInterp Map.empty natFormula1
print $ "∃x ∈ ℕ. even(x) = " ++ show natResult1

let natFormula2 = Forall "x" (Pred ">=" [Var "x", Const 0])  -- ∀x ∈ ℕ. x ≥ 0
let natResult2 = evalFormula naturalsInterp Map.empty natFormula2
print $ "∀x ∈ ℕ. x ≥ 0 = " ++ show natResult2

let natFormula3 = Exists "x" (Pred "==" [Var "x", Const 100])  -- ∃x ∈ ℕ. x = 100
let natResult3 = evalFormula naturalsInterp Map.empty natFormula3
print $ "∃x ∈ ℕ. x = 100 = " ++ show natResult3

## Step 9: Probability Example

Simple probabilistic reasoning over finite domains.

In [ ]:
-- Simple probability over finite domain
data ProbValue = Certain Bool | Probabilistic [(Bool, Double)]
  deriving Show

-- Probabilistic version of truth values
instance TwoMonBLat ProbValue where
  top = Certain True
  bot = Certain False
  neg (Certain b) = Certain (not b)
  neg (Probabilistic dist) = Probabilistic [(not b, p) | (b, p) <- dist]
  conj (Certain b1) (Certain b2) = Certain (b1 && b2)
  conj (Certain True) (Probabilistic dist) = Probabilistic dist
  conj (Certain False) (Probabilistic _) = Certain False
  conj (Probabilistic dist) (Certain True) = Probabilistic dist
  conj (Probabilistic _) (Certain False) = Certain False
  conj (Probabilistic d1) (Probabilistic d2) = 
    Probabilistic [(b1 && b2, p1 * p2) | (b1, p1) <- d1, (b2, p2) <- d2]
  disj = undefined  -- Simplified for now
  implies = undefined  -- Simplified for now

-- Example: Fair coin
coinInterp :: Interpretation Double
coinInterp = Interpretation {
  domain = Finite [0.0, 1.0],  -- 0 = heads, 1 = tails
  functions = Map.fromList [],
  predicates = Map.fromList [
    ("heads", \[x] -> x == 0),
    ("tails", \[x] -> x == 1)
  ]
}

print "Probability example framework defined (simplified)"